# Overview

**ETL** : Extract, Transform, Load Project

**Extract**: extract data from different sources, read the data, and store into a dataframe.

**Transform**: clean, structure, and transform the raw data that has been extracted from the sources into a specific schema, so it meets the operation needs.

**Load**: load and store the data into a database for future analysis.

Data Source:

1. Energy Data: https://www.kaggle.com/datasets/lislejoem/us_energy_census_gdp_10-14?resource=download

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# Extract

## Store CSV into DataFrame

In [3]:
#Load in energy data
csv_file_1 = "resources/energy_census_and_economic_data_US_2010-2014.csv"
energy_df = pd.read_csv(csv_file_1)
energy_df.head()

,StateCodes,State,Region,Division,Coast,Great Lakes,TotalC2010,TotalC2011,TotalC2012,TotalC2013,...,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014
0,AL,Alabama,3.0,6.0,1.0,0.0,1931522,1905207,1879716,1919365,...,1.165832,1.157861,-0.020443,-0.168414,0.396416,0.420102,1.011941,1.001333,1.562247,1.577963
1,AK,Alaska,4.0,9.0,1.0,0.0,653221,653637,649341,621107,...,3.203618,2.869760,-1.175137,-1.949571,-3.789313,-13.754494,0.948185,1.835376,-0.585695,-10.884734
2,AZ,Arizona,4.0,8.0,0.0,0.0,1383531,1424944,1395839,1414383,...,1.090035,1.091283,1.341472,-0.420875,-0.580562,-1.313050,2.317801,0.621971,0.509473,-0.221767
3,AR,Arkansas,3.0,7.0,0.0,0.0,1120632,1122544,1067642,1096438,...,2.141877,2.129805,1.369514,5.131282,3.910476,6.280636,3.336628,7.155212,6.052353,8.410441
4,CA,California,4.0,9.0,1.0,0.0,7760629,7777115,7564063,7665241,...,4.207353,4.177389,-1.162079,-1.173951,-1.341226,-0.830982,2.761377,2.772770,2.866127,3.346406


In [4]:
# energy_columns = energy_df.columns

# for column in energy_columns:
#     print(column)

In [5]:
# Calculate yearly GDP rates and adding the columns into the energy_df dataframe.
# yearly GDP change. Calculated by dividing the second year’s GDP by the first year’s GDP.
gdp_2010_2011 = []
gdp_2011_2012 = []
gdp_2012_2013 = []
gdp_2013_2014 = []

i = 0
while i < len(energy_df):
    if True:
        gdp_2010_2011.append(energy_df["GDP2011"][i] / energy_df["GDP2010"][i] * 100)
        gdp_2011_2012.append(energy_df["GDP2012"][i] / energy_df["GDP2011"][i] * 100)
        gdp_2012_2013.append(energy_df["GDP2013"][i] / energy_df["GDP2012"][i] * 100)
        gdp_2013_2014.append(energy_df["GDP2014"][i] / energy_df["GDP2013"][i] * 100)
    i += 1
    
energy_df["GDP_2010_2011"] = gdp_2010_2011
energy_df["GDP_2011_2012"] = gdp_2011_2012
energy_df["GDP_2012_2013"] = gdp_2012_2013
energy_df["GDP_2013_2014"] = gdp_2013_2014
    
gdp_df = energy_df[['StateCodes', 'State', 'GDP_2010_2011', 'GDP_2011_2012', 'GDP_2012_2013', 'GDP_2013_2014' ]]
gdp_df = gdp_df.rename(columns={'StateCodes':'statecodes', 'State':'state', 'GDP_2010_2011': 'gdp_2010_2011', 'GDP_2011_2012': 'gdp_2011_2012', 'GDP_2012_2013': 'gdp_2012_2013', 'GDP_2013_2014': 'gdp_2013_2014' }).copy()
gdp_df

,statecodes,state,gdp_2010_2011,gdp_2011_2012,gdp_2012_2013,gdp_2013_2014
0,AL,Alabama,103.235856,102.946439,102.307734,103.094513
1,AK,Alaska,109.402896,103.869383,97.203941,96.954455
2,AZ,Arizona,103.350746,104.111662,101.856888,103.868430
3,AR,Arkansas,104.240497,101.977313,104.581723,103.783278
4,CA,California,103.590774,104.443085,104.413139,104.954967
5,CO,Colorado,103.350013,104.066961,104.020306,105.733114
6,CT,Connecticut,100.830666,102.605365,101.718157,103.443137
7,DE,Delaware,103.622965,101.691675,101.068709,106.611449
8,FL,Florida,100.751307,103.998841,104.276230,104.795298
9,GA,Georgia,102.675864,103.833998,103.668176,104.645031


# Transform

In [6]:
#Cleaning Energy Consumption Data
year = []
state = []
#Total energy consumption:
energy_consumption = []
#Sectors of energy consumption:
biomass_consumption = []
coal_consumption = []
elec_consumption = []
fossfuel_consumption = []
natgas_consumption = []

#GDP by year
gdp = []

year_starter = 2010
year_counter = 0


while True:
    for index, row in energy_df.iterrows():
        if (index == (len(energy_df)-1)):
            year.append(year_starter + year_counter)
            state.append(energy_df['State'][index])
            energy_consumption.append(energy_df[f"TotalC{year_starter + year_counter}"][index])
            biomass_consumption.append(energy_df[f"BiomassC{year_starter + year_counter}"][index])
            print(f"BiomassC{year_starter + year_counter}")
            coal_consumption.append(energy_df[f"CoalC{year_starter + year_counter}"][index])
            elec_consumption.append(energy_df[f"ElecC{year_starter + year_counter}"][index])
            fossfuel_consumption.append(energy_df[f"FossFuelC{year_starter + year_counter}"][index])
            natgas_consumption.append(energy_df[f"NatGasC{year_starter + year_counter}"][index])
            gdp.append(energy_df[f"GDP{year_starter + year_counter}"][index])
            year_counter += 1
            continue

        else:
            year.append(year_starter + year_counter)
            state.append(energy_df['State'][index])
            energy_consumption.append(energy_df[f"TotalC{year_starter + year_counter}"][index])
            biomass_consumption.append(energy_df[f"BiomassC{year_starter + year_counter}"][index])
            coal_consumption.append(energy_df[f"CoalC{year_starter + year_counter}"][index])
            elec_consumption.append(energy_df[f"ElecC{year_starter + year_counter}"][index])
            fossfuel_consumption.append(energy_df[f"FossFuelC{year_starter + year_counter}"][index])
            natgas_consumption.append(energy_df[f"NatGasC{year_starter + year_counter}"][index])
            gdp.append(energy_df[f"GDP{year_starter + year_counter}"][index])
    
    if (year_starter + year_counter == 2015):
        break

BiomassC2010
BiomassC2011
BiomassC2012
BiomassC2013
BiomassC2014


In [7]:
energy_df_cleaned = pd.DataFrame({
#     'year_state': year_state,
    'year': year,
    'state': state,
    'total_energy': energy_consumption ,
    'gdp': gdp,
    'biomass_consumption': biomass_consumption,
    'coal_consumption': coal_consumption,
    'electricity_consumption': elec_consumption,
    'fossil_fuel_consumption': fossfuel_consumption,
    'natural_gas_consumption': natgas_consumption
})

energy_df_cleaned

,year,state,total_energy,gdp,biomass_consumption,coal_consumption,electricity_consumption,fossil_fuel_consumption,natural_gas_consumption
0,2010,Alabama,1931522,176220.75,169088,718684,310023,1785688,544405
1,2010,Alaska,653221,54220.00,4178,14548,21315,634777,334978
2,2010,Arizona,1383531,247333.00,29289,457909,248506,1292346,336208
3,2010,Arkansas,1120632,104928.75,94865,293689,164439,909827,274774
4,2010,California,7760629,1960935.00,280124,54972,882107,5741492,2325411
...,...,...,...,...,...,...,...,...,...
255,2014,West Virginia,752942,74432.75,30091,816460,111559,1163308,161661
256,2014,Wisconsin,1868867,293341.25,136809,417096,237116,1437711,477923
257,2014,Wyoming,535612,40875.50,4566,489300,58461,803967,141763
258,2014,District of Columbia,178929,116539.25,920,48,38193,55217,35316


In [8]:
energy_df_cleaned.columns

Index(['year', 'state', 'total_energy', 'gdp', 'biomass_consumption',
       'coal_consumption', 'electricity_consumption',
       'fossil_fuel_consumption', 'natural_gas_consumption'],
      dtype='object')

## Create database connection

In [9]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'energy_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [10]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

['energy_consumption', 'minimum_wage']

## Load DataFrames into database

In [11]:
energy_df_cleaned.to_sql(name='energy_consumption', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "biomass_consumption" of relation "energy_consumption" does not exist
LINE 1: ...ergy_consumption (year, state, total_energy, gdp, biomass_co...
                                                             ^

[SQL: INSERT INTO energy_consumption (year, state, total_energy, gdp, biomass_consumption, coal_consumption, electricity_consumption, fossil_fuel_consumption, natural_gas_consumption) VALUES (%(year)s, %(state)s, %(total_energy)s, %(gdp)s, %(biomass_consumption)s, %(coal_consumption)s, %(electricity_consumption)s, %(fossil_fuel_consumption)s, %(natural_gas_consumption)s)]
[parameters: ({'year': 2010, 'state': 'Alabama', 'total_energy': 1931522, 'gdp': 176220.75, 'biomass_consumption': 169088, 'coal_consumption': 718684, 'electricity_consumption': 310023, 'fossil_fuel_consumption': 1785688, 'natural_gas_consumption': 544405}, {'year': 2010, 'state': 'Alaska', 'total_energy': 653221, 'gdp': 54220.0, 'biomass_consumption': 41

In [ ]:
cleaned_min_wage_data_df.to_sql(name='minimum_wage', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the DB table

In [ ]:
pd.read_sql_query('select * from energy_consumption', con=engine).head()

In [ ]:
pd.read_sql_query('select * from minimum_wage', con=engine).head()